# Análisis para tienda de alimentos
## Objetivo:
Estudiar el comportamiento de los usuarios para la aplicación de la empresa.

### Objetivos específicos:
* Análisis exploratorio:
  * Número de eventos y usuarios en los registros
  * Promedio de eventos por usuario
  * Periodo de los datos
  * Confirmación de 3 grupos experimentales

* Análisis de embudo:
  * Elaborar el embudo
  * Proporción de usuarios por cada evento
  * Proporción de usuarios que pasan a la siguiente etapa
  * Evaluar las etapas
  * Cantidad de usuarios que completan el viaje (primer evento hasta pago)

* Análisis de experimento A/A/B:
  * Número de usuarios por grupo
  * Evaluar diferencia entre grupos control (A/A)
  * Evaluar diferencia entre eventos para los grupos control
  * Comparar diferencia de grupos control con grupo con variante

## Diccionario de datos:
* ``EventName``: nombre del evento.
* ``DeviceIDHash``: identificador de usuario unívoco.
* ``EventTimestamp``: hora del evento.
* ``ExpId``: número de experimento. 246 y 247 son los grupos de control, y 248 es el grupo de prueba.